In [2]:
datadir="/home/qjh/anomaly_data/datasets/mvtec"
texture_source_dir="/home/qjh/anomaly_data/datasets/dtd/images"
savedir = "./saved_model"

In [3]:
from omegaconf import OmegaConf
import torch
from main import run

config_dict = {
    "EXP_NAME": "MemSeg",
    "SEED": 42,
    "DATASET": {
        "datadir": datadir,
        "texture_source_dir": texture_source_dir,
        "anomaly_mask_info": "./anomaly_mask.json",
        "target": "my_bottle",
        "resize": [288, 288],
        "imagesize": 256,
        "structure_grid_size": 8,
        "transparency_range": [0.15, 1.0],
        "perlin_scale": 6,
        "min_perlin_scale": 0,
        "perlin_noise_threshold": 0.5,
    },
    
    "DATALOADER": {
        "batch_size": 8,
        "num_workers": 0
    },
    
    "MEMORYBANK": {
        "nb_memory_sample": 30
    },
    
    "MODEL": {
        "feature_extractor_name": "resnet18"
    },
    
    "TRAIN": {
        "batch_size": 6,
        "num_training_steps": 2000,
        "l1_weight": 0.6,
        "focal_weight": 0.4,
        "focal_alpha": None,
        "focal_gamma": 4,
        "use_mlflow": True
    },
    
    "OPTIMIZER": {
        "lr": 0.003,
        "weight_decay": 0.0005
    },
    
    "SCHEDULER": {
        "min_lr": 0.0001,
        "warmup_ratio": 0.1,
        "use_scheduler": True
    },
    
    "LOG": {
        "log_interval": 1,
        "eval_interval": 100
    },
    
    "RESULT": {
        "savedir": savedir
    }
}

cfg = OmegaConf.create(config_dict)

In [4]:
target_specific_config = {
    "hazelnut": {
        "use_mask"     : True,
        "bg_threshold" : 50,
        "bg_reverse"   : True
    },
    "leather": {
        "use_mask"     : False,
        "bg_threshold" : None,
        "bg_reverse"   : None
    },
    "metal_nut": {
        "use_mask"     : True,
        "bg_threshold" : 40,
        "bg_reverse"   : True
    },
    "tile": {
        "use_mask"     : False,
        "bg_threshold" : None,
        "bg_reverse"   : None
    },
    "wood": {
        "use_mask"     : False,
        "bg_threshold" : None,
        "bg_reverse"   : None
    },
    "grid": {
        "use_mask"     : False,
        "bg_threshold" : None,
        "bg_reverse"   : None
    },
    "cable": {
        "use_mask"     : False,
        "bg_threshold" : 150,
        "bg_reverse"   : True
    },
    "capsule": {
        "use_mask"     : True,
        "bg_threshold" : 120,
        "bg_reverse"   : False
    },
    "transistor": {
        "use_mask"     : True,
        "bg_threshold" : 90,
        "bg_reverse"   : False
    },
    "carpet": {
        "use_mask"     : False,
        "bg_threshold" : None,
        "bg_reverse"   : None
    },
    "bottle": {
        "use_mask"     : True,
        "bg_threshold" : 250,
        "bg_reverse"   : False
    },
    "screw": {
        "use_mask"     : True,
        "bg_threshold" : 110,
        "bg_reverse"   : False
    },
    "zipper": {
        "use_mask"     : True,
        "bg_threshold" : 100,
        "bg_reverse"   : False
    },
    "pill": {
        "use_mask"     : True,
        "bg_threshold" : 100,
        "bg_reverse"   : True
    },
    "toothbrush": {
        "use_mask"     : True,
        "bg_threshold" : 30,
        "bg_reverse"   : True
    }
}

In [ ]:
cfg.DATASET = OmegaConf.merge(
    cfg.DATASET,
    target_specific_config[cfg.DATASET.target]
)

assert cfg.DATASET.target is not None, "Please set DATASET.target"

run(cfg)

torch.cuda.empty_cache()

In [ ]:
from IPython.display import Image, display
display(Image("saved_model/MemSeg-bottle/combined_sample_6.png"))
display(Image("saved_model/MemSeg-bottle/combined_sample_7.png"))
display(Image("saved_model/MemSeg-bottle/combined_sample_8.png"))
display(Image("saved_model/MemSeg-bottle/combined_sample_9.png"))

In [ ]:
import mlflow
!mlflow ui --backend-store-uri file:'./mlruns'